### импортируем все нужное

In [1]:
import re
import pandas as pd
import csv
from tqdm import tqdm
from redditscore.tokenizer import CrazyTokenizer

In [2]:
from nltk.corpus import stopwords
stops = stopwords.words('russian')

## Расширяем список стопслов

In [3]:
stops.extend(stopwords.words('english'))

In [4]:
badwords = [
'я', 'а', 'да', 'но', 'тебе', 'мне', 'ты', 'и', 'у', 'на', 'ща', 'ага',
'так', 'там', 'какие', 'который', 'какая', 'туда', 'давай', 'короче', 'кажется', 'вообще',
'ну', 'не', 'чет', 'неа', 'свои', 'наше', 'хотя', 'такое', 'например', 'кароч', 'как-то',
'нам', 'хм', 'всем', 'нет', 'да', 'оно', 'своем', 'про', 'вы', 'тд', 'тп', 'т.д', 'т.п',
'вся', 'вам', 'это', 'эта', 'эти', 'этот', 'прям', 'либо', 'как', 'мы',
'просто', 'блин', 'очень', 'самые', 'твоем', 'ваша', 'кстати', 'вроде', 'типа', 'пока', 'ок',
    '°', '͜ʖ', '͡', 'ツ', 'ʕ•ᴥ•ʔ'
]

In [5]:
stops.extend(badwords)

In [6]:
print(stops)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

### Открываем данные

In [7]:
with open('../Data/task2_ru_training.tsv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep="\t")

In [8]:
df.head(5)

,tweet_id,tweet,class
0,760402871867367424,"Настало время для ингаляторов. Дружок, Сальбут...",0
1,1035908416869462016,15) На прошлой зимней олимпиаде большинство лы...,1
2,1089839736427032577,Не соглашусь с заменой ЗОК на метопролол в так...,0
3,779671488748224513,"@di2m1 мезим Смекта Если отравление, то лоперамид",0
4,738309299756240897,Уберите микроволновки и имодиум Действуют соу...,0


### выбираем тестовые твиты

In [9]:
test_data = [df['tweet'][8],
             df['tweet'][9], # дефисы
             df['tweet'][19],
             df['tweet'][268], # дефисы, капс
             
             df['tweet'][273], # латиница, нет пробелов вокруг скобки
             
             df['tweet'][10], # 0,5г ; р/д
             df['tweet'][20], # 125 мгк 2 раза ; Вентолин 100 мгк
            
             df['tweet'][2368], # нет пробела между предложениями
             
             df['tweet'][14], # кавычки, юзернейм, эмоджи
             df['tweet'][18], # хештеги, юзернейм
             df['tweet'][25], # смайл, юзернейм, эмоджи
             df['tweet'][116], # эмоджи
             df['tweet'][270], # смайл
             
             df['tweet'][232] # сокращенная ссылка
            ]

In [10]:
#for tweet in test_data:
#    print(tweet)

## Настраиваем токенизатор

In [11]:
tokenizer = CrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='splits', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=True, neg_emojis=True, neutral_emojis=True, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=True, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=stops # удаляет стоп слова (разобраться почему падает на кастомном списке)
                          )

## Добавляем еще несколько удалений и замен

In [72]:
def process(tweet, log = False) -> str:
    tweet = re.sub('(\w)\.([А-ЯЁ])', r'\1. \2', tweet) # ставим пробелы
    tweet = re.sub('-', ' ', tweet) # удаляем меняем деффисы на пробел
    if log:
        #print('!до токенизации')
        print(tweet)
    # токенизируем (см. удаления и замены выше)
    tokens = tokenizer.tokenize(tweet)
    if log:
        #print('!после токенизации')
        print(tokens)
        
    new_tokens = []
    for token in tokens:
        if re.search('\d', token): # удаляем все токены, содержащие цифры
            continue
        elif re.search('[!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', token): #удаляем все токены, содержащие оставшиеся символы
            continue
        elif len(token) < 2:
            continue
        new_tokens.append(token)
            
    if log:
        #print('!после обработки каждого токена')
        print(new_tokens)
        #print('!output')
    
    return ' '.join(new_tokens)

In [84]:
res = []
for tweet in test_data:
    print(tweet)
    tweet = process(tweet, log=False)
    res.append(tweet)
    print(tweet, '\n')

Все эти слюнтяи, утверждающие, будто суицид-удел слабых людей, найдут тысячу причин не глотать горсть прозака.Им просто не хватает смелости.
слюнтяи утверждающие суицид удел слабых людей найдут тысячу причин глотать горсть прозака хватает смелости 

Это вам не лыжи,сальбутамол не поможет. Россия - Норвегия 5:1
лыжи сальбутамол поможет россия норвегия 

Если оланзапин мне не поможет, тогда мне назначат галоперидол. Не хочу докатываться до галоперидола. Надо что-то делать с этой проклятой тревожностью.
оланзапин поможет назначат галоперидол хочу докатываться галоперидола делать проклятой тревожностью 

из-за флуоксетина катастрофически не хочется что-либо есть, но почему-то БЛИНЫ ИЗ ТЕРЕМКА в меня влезают всегда  Теремок, спасибо что живая
флуоксетина катастрофически хочется почему БЛИНЫ ТЕРЕМКА влезают теремок спасибо живая 

В моей голове Хенвон любит lil peep; перкосет, молли, перпл дранк(это все наркота, если кто не в курсе); организует нелегальные вечеринки.
моей голове хенвон любит

In [85]:
res

['слюнтяи утверждающие суицид удел слабых людей найдут тысячу причин глотать горсть прозака хватает смелости',
 'лыжи сальбутамол поможет россия норвегия',
 'оланзапин поможет назначат галоперидол хочу докатываться галоперидола делать проклятой тревожностью',
 'флуоксетина катастрофически хочется почему БЛИНЫ ТЕРЕМКА влезают теремок спасибо живая',
 'моей голове хенвон любит lil peep перкосет молли перпл дранк наркота курсе организует нелегальные вечеринки',
 'ципрофлоксацин смысл жизни',
 'ветеринар прописал ингаляции фликсотидом мгк раза сутки прописал вентолин мгк случай приступов',
 'день выпила таблеток алпразолама чувствую расслабленно ох днем часа спала',
 'ЮЗЕРНЕЙМ лечатся футболисты астмы синдрома дефицита внимания столь любимыми нероссийских спортсменов законными амфетамином декстроамфетамином',
 'ЮЗЕРНЕЙМ порекомендовал сотрудникам wada выпить мельдония вдохнуть сальбутамола попробовать приблизиться результату любого медалиста splits вада крушельницкий',
 'ЮЗЕРНЕЙМ питер мем

## Лемматизация

In [86]:
from pymystem3 import Mystem
mstm = Mystem()

In [90]:
lem = []
for tweet in tqdm(res):
    lem.append(''.join(mstm.lemmatize(tweet)).strip())

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:11<00:00,  1.24it/s]


In [91]:
lem

['слюнтяй утверждать суицид удел слабый человек находить тысяча причина глотать горсть прозак хватать смелость',
 'лыжа сальбутамол помогать россия норвегия',
 'оланзапина помогать назначать галоперидол хотеть докатываться галоперидол делать проклятый тревожность',
 'флуоксетин катастрофически хотеться почему блин теремок влезать теремок спасибо живой',
 'мой голова хенвон любить lil peep перкосать молль перпл дранк наркота курс организовывать нелегальный вечеринка',
 'ципрофлоксацин смысл жизнь',
 'ветеринар прописывать ингаляция фликсотид мгк раз сутки прописывать вентолина мгк случай приступ',
 'день выпивать таблетка алпразолам чувствовать расслабленно ох день час спать',
 'юзернейм лечиться футболист астма синдром дефицит внимание столь любимый нероссийский спортсмен законный амфетамин декстроамфетамин',
 'юзернейм порекомендовать сотрудник wada выпивать мельдония вдыхать сальбутамол попробовать приближаться результат любой медалист splits вад крушельницкий',
 'юзернейм питер мема

# Полный скрипт препроцессинга

In [96]:
def write_data(out_file, prep = True, lemmatize = 'mystem'):
    with open('../Data/task2_ru_training.tsv', 'r', encoding='utf-8') as f:
        df = pd.read_csv(f, sep="\t")
    
    if prep:
        new_tweets = []
        for tweet in tqdm(df['tweet']):
            new_tweets.append(process(tweet, log=False))
        df['tweet'] = new_tweets
        
    if lemmatize == 'mystem':
        lem = []
        for tweet in tqdm(df['tweet']):
            lem.append(''.join(mstm.lemmatize(tweet)).strip())
        df['tweet'] = lem


    with open('../Data/{}'.format(out_file), 'w', encoding='utf-8') as output:
        df.to_csv(output, sep='\t', index=False, line_terminator='\n')

    print('Done! Your data is in {}'.format(out_file))

In [97]:
out_file = 'mystem_lem.tsv'

In [98]:
write_data(out_file, prep = True, lemmatize = 'mystem')

100%|████████████████████████████████████████████████████████████████████████████| 6090/6090 [1:48:36<00:00,  1.10s/it]


Done! Your data is in mystem_lem.tsv
